In [1]:
import pandas as pd
import numpy as np

In [2]:
import tensorflow as tf
import keras

In [3]:
# load data

data = pd.read_csv('eng_-french.csv')
data.head()

,English words/sentences,French words/sentences
0,Hi.,Salut!
1,Run!,Cours !
2,Run!,Courez !
3,Who?,Qui ?
4,Wow!,Ça alors !


In [4]:
data.shape

(175621, 2)

In [5]:
data.columns

Index(['English words/sentences', 'French words/sentences'], dtype='object')

In [6]:
len(np.unique(data['English words/sentences']))

123100

In [7]:
'Run ?'.split(' ')

['Run', '?']

In [8]:
def uniqueTokens(data):
    dict_Tokens = {}
    length = []
    for k in data.columns:
        dataList = []
        for b in data[k]:
            for m in b.split(' '):
                length.append(len(b.split(' ')))
                if m.lower() not in dataList:
                    dataList.append(m.lower())
                else:
                    pass
        dict_Tokens[k+ ' Tokens'] = dataList
    return dict_Tokens, max(length)

DictTokens, max_length = uniqueTokens(data)

In [9]:
data.columns

Index(['English words/sentences', 'French words/sentences'], dtype='object')

In [10]:
DictTokens.keys()

dict_keys(['English words/sentences Tokens', 'French words/sentences Tokens'])

In [11]:
len(DictTokens['English words/sentences Tokens'])

25639

In [12]:
len(DictTokens['French words/sentences Tokens'])

46416

In [13]:
max_length

55

In [14]:
x,y = data['English words/sentences'], data['French words/sentences']

In [15]:
# lowercase all letters
x_lower = x.apply(lambda x:x.lower())
y_lower = y.apply(lambda x:x.lower())

In [16]:
import re
x_lower = x_lower.apply(lambda x:re.sub("'",'',x))
y_lower = y_lower.apply(lambda x:re.sub("'",'',x))

In [17]:
import string
exclude = set(string.punctuation) # set of all special characters
#remove all the special characters
x_lower = x_lower.apply(lambda x:''.join(ch for ch in x if ch not in exclude))
y_lower = y_lower.apply(lambda x:''.join(ch for ch in x if ch not in exclude))

In [18]:
# remove all numbers from text
digits = string.digits
remove_digits = str.maketrans('','',digits)
x_lower = x_lower.apply(lambda x:x.translate(remove_digits))
y_lower = y_lower.apply(lambda x:x.translate(remove_digits))

In [19]:
# remove extra spaces
x_lower = x_lower.apply(lambda x: x.strip())
y_lower = y_lower.apply(lambda x: x.strip())

In [20]:
# add start and end tokens to target sequences
y_lower = y_lower.apply(lambda x: 'START_ '+x +' _END')

In [21]:
y_lower[:2]

0    START_ salut _END
1    START_ cours _END
Name: French words/sentences, dtype: object

In [23]:
# get english and igbo vocabulary
all_eng_words = set()
for eng in x_lower:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
            
            
all_french_words = set()

for igb in y_lower:
    for word in igb.split():
        if word not in all_igbo_words:
            all_igbo_words.add(word)

In [24]:
print(len(all_eng_words))
print(len(all_french_words))

14409
0


In [ ]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_igbo_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_igbo_words)
num_encoder_tokens, num_decoder_tokens